In [73]:
%load_ext autoreload
%autoreload 2
import numpy as np
from tqdm import tqdm

from seq.model import Chunk, ChunkDatabase, Cluster

np.random.seed(42)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [49]:
data = np.random.randint(1, 5, (100_000, 32))
db = ChunkDatabase(data)

In [50]:
chunks = db.chunks

In [51]:
processed_chunks = set()
clusters: list[Cluster] = []

for chunk in chunks:
  if chunk in processed_chunks:
    continue

  cluster = Cluster(chunk)
  left = cluster.left
  right = cluster.right

  left_candidate = [
    c
    for c in db.get_candidate(cluster.left)
    if c not in processed_chunks and cluster.can_appendleft(c)
  ]

  right_candidate = [
    c
    for c in db.get_candidate(cluster.right)
    if c not in processed_chunks and cluster.can_append(c)
  ]

  while len(left_candidate) > 0 or len(right_candidate) > 0:
    if len(left_candidate):
      cluster.appendleft(left_candidate[0])
      processed_chunks.add(left_candidate[0])
    if len(right_candidate):
      cluster.append(right_candidate[0])
      processed_chunks.add(right_candidate[0])

    left_candidate = [
      c
      for c in db.get_candidate(cluster.left)
      if c not in processed_chunks and cluster.can_appendleft(c)
    ]

    right_candidate = [
      c
      for c in db.get_candidate(cluster.right)
      if c not in processed_chunks and cluster.can_append(c)
    ]

  clusters.append(cluster)

In [59]:
print(len(clusters))
clusters = sorted(clusters, key=lambda c: c.size, reverse=True)

113


In [60]:
canvas = np.zeros((data.shape[0] * 100, data.shape[1]))
canvas.shape

(10000000, 32)

In [72]:
clusters[0][0]

Chunk([4 3 2], 14, 17, #46898)

In [7]:
canvas = np.zeros((data.shape[0] * 100, data.shape[1]))
positions = []
for cluster in tqdm(clusters, desc="Processing Clusters..."):
  start = cluster.start
  end = cluster.end
  subcanvas = canvas[:, start:end]

  cluster.draw(subcanvas, db.min_n_chunks)

  positions.append(position)
  if position != -1:
    canvas[position : position + shape.shape[0]][shape != 0] = shape[shape != 0]

Processing paths: 100%|██████████| 340/340 [01:23<00:00,  4.09it/s]


In [8]:
filtered_canvas = canvas[~np.all(canvas == 0, axis=1)]
print(canvas.shape)
print(filtered_canvas.shape)

(10000000, 32)
(5023629, 32)


In [9]:
np.sum(filtered_canvas != 0) / np.prod(filtered_canvas.shape)

np.float64(0.4317462908785661)

In [10]:
def print_row(row: np.ndarray):
  print("".join([f"{int(x)}" if x != 0 else " " for x in row]))

In [13]:
rows = []
prev_row = np.zeros(canvas.shape[1])
for i in range(0, canvas.shape[0]):
  row = canvas[i]
  if not np.array_equal(row, prev_row) and not np.sum(row) == 0:
    rows.append(row)
    print_row(row)
    prev_row = row

3413  11 2 423 1134   344 313311
3413  11 2 423   3443 344 313311
3413  11 2 423 1434   344 313311
3413  11 2 423 14     344 313311
3413  11 2 423 14   21344 313311
3413  11 2 423 14 4321344 313311
3413  11 2 423 14 4321 442313311
3413  11 2     14 4321 442313311
3413  11 2     14 4321 442313   
3413  11 2     14 4321 44231    
3413  11 2333  14 4321 44231331 
3413  11 2333  14 4321244 31331 
3413  11 2     14 4321244 31331 
3413  11 2     14 4321244 31    
3413  11 2 134 14 4321244 31441 
3413  11 2 134 14 432124  31441 
3413  11 2 134 14 432124  31    
3413  11 2     14 432124  31    
3413  11 2114  14 432124  313311
3413  11 2114  14 432124  31    
3413  11 2114  14 432124  314411
3413  11 2114  14 432124  31    
3413  11 2114  14 432124  312131
3413  11 2114  14 432124  31    
3413     2114  14 432124  31    
34133    2114  14 432124  31    
34133 1132114  14 432124  311421
34133 1132     14 432124  311421
34133 11322322 14 432124  311421
34133 11322322 14 432124  31    
34133 1132

In [ ]:
len(rows)

1040